In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install unidecode

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from re import sub
from unidecode import unidecode
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class ws_mercadolivre:
  def __init__(self):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    self.driver = webdriver.Chrome(options=options)
    self.driver.get('https://www.mercadolivre.com.br/')

    
  def get_url(self, url):
    self.driver.get(url)
 

  def print_url(self):
    print(self.driver.current_url)


  def close(self):
    self.driver.quit()


  def search(self, text):
    search_xpath = '//input[@class="nav-search-input"]'
    element_search = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, search_xpath)))
    element_search.send_keys(text)
    sleep(0.2)
    element_search.send_keys(Keys.ENTER)


  def classificador(self, text, rest_sim, rest_nao):
    text = sub('[!@.),#$(]+', '', text)
    text = unidecode(text, "utf-8")
    text_list = text.lower().split(' ')
    num_sim = 0
    num_nao = 0
    for item_text in text_list:
      for item_sim in rest_sim:
        if item_text == item_sim:
          num_sim += 1
      for item_nao in rest_nao:
        if item_text == item_nao:
          num_nao += 1

    if num_nao == 0 and num_sim == len(rest_sim):
      return True
    else:
      return False


  def titulos_pesquisa(self):
    title_xpath = '//a[@class="ui-search-item__group__element ui-search-link"]'
    elements_title = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, title_xpath)))
    titulos = []
    for element in elements_title:
      titulos.append(element.text)

    return titulos


  def get_links(self, rest_sim, rest_nao):
    link_xpath = '//a[@class="ui-search-item__group__element ui-search-link"]'
    elements_links = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, link_xpath)))
    titulos = self.titulos_pesquisa()
    links = []
    for cont in range(len(titulos)):
      status = self.classificador(titulos[cont], rest_sim, rest_nao)
      if status:
        links.append(elements_links[cont].get_attribute('href').split('#')[0])
    
    return links

  def next_page(self):
    #next_path = '//a[@class="andes-pagination__link ui-search-link"]'
    try:
      next_page = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.LINK_TEXT, 'Seguinte')))
      self.driver.execute_script("arguments[0].click();", next_page)
      sleep(1)
    except:
      return False
    return True
    


  def titulo_anuncio(self):
    title_xpath = '//div[@class="ui-pdp-header__title-container"]/h1[@class="ui-pdp-title"]'
    elements_title = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, title_xpath)))
    return elements_title.text


  def preco(self):
    reais_xpath = '//span[@class="andes-money-amount ui-pdp-price__part andes-money-amount--cents-superscript andes-money-amount--compact"]/meta[@itemprop="price"]'
    reais_element = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, reais_xpath)))
    return float(reais_element.get_attribute("content"))


  def marca(self):
    try:
      verificacao = '//tr[@class="andes-table__row"]/th[@class="andes-table__header andes-table__header--left ui-pdp-specs__table__column ui-pdp-specs__table__column-title"]'
      element_verificacao = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, verificacao)))
      for cont in range(len(element_verificacao)):
        if element_verificacao[cont].text.upper() == 'MARCA':
          mark_xpath = '//td[@class="andes-table__column andes-table__column--left ui-pdp-specs__table__column"]/span[@class="andes-table__column--value"]'
          element_mark = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, mark_xpath)))[cont]
          text = element_mark.text.upper()
          text = sub('[!@.),#$(]+', '', text)
          return text
    except:
      return np.nan

  def numero_peca(self):
    try:
      verificacao = '//tr[@class="andes-table__row"]/th[@class="andes-table__header andes-table__header--left ui-pdp-specs__table__column ui-pdp-specs__table__column-title"]'
      element_verificacao = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, verificacao)))
      for cont in range(len(element_verificacao)):
        if element_verificacao[cont].text.upper() == 'NÚMERO DE PEÇA':
          mark_xpath = '//td[@class="andes-table__column andes-table__column--left ui-pdp-specs__table__column"]/span[@class="andes-table__column--value"]'
          element_mark = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, mark_xpath)))[cont]
          text = element_mark.text.upper()
          text = sub('[!@.),#$(]+', '', text)
          return text
    except:
      return np.nan

  def vendas(self):
    vendas_xpath = '//div[@class="ui-pdp-header__subtitle"]/span[@class="ui-pdp-subtitle"]'
    element_vendas = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, vendas_xpath)))
    text_list = element_vendas.text.split(' ')
    for word in text_list:
      try:
        vendas_peca = int(word)
        return vendas_peca
      except:
        continue


  def estoque(self):
    try:
      estoque_xpath = '//span[@class="andes-button__content"]/span[@class="ui-pdp-buybox__quantity__available"]'
      element_estoque = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, estoque_xpath)))
      return int(element_estoque.text[1:3])
    except:
      estoque_xpath = '//div[@class="ui-pdp-buybox__quantity"]/p[@class="ui-pdp-color--BLACK ui-pdp-size--MEDIUM ui-pdp-family--SEMIBOLD"]'
      element_estoque = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, estoque_xpath)))
      return 1
    else:
      return 0


  def desc(self):
    try:
      desc_xpath = '//div[@class="ui-pdp-description"]/p[@class="ui-pdp-description__content"]'
      element_desc = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, desc_xpath)))
      return element_desc.text
    except:
      return np.nan

In [ ]:
drive_01 = ws_mercadolivre()
drive_01.print_url()
drive_01.search('Tomada 7 Polos Carreta Macho')


rest_sim = ['iphone 10']
rest_nao = ['sansung', 'lg', 'xaiomi']

links = []
status = True

while status:
  drive_01.print_url()
  links.extend(drive_01.get_links(rest_sim, rest_nao))
  status = drive_01.next_page()


print('-'*100)
print(links)
print('-'*100)

print('\n\n')

data = datetime.today().strftime('%Y-%m-%d')
df_dict = {'Data': data, 'Titulo': [], 'Preco':[], 'Posicao':[], 
           'Vendas_Anuncio': [], 'Estoque': [], 'Marca':[], 
           'Descricao': [], 'Link':[]}

for contador in range(len(links)):
  drive_01.get_url(links[contador])
  sleep(np.random.uniform(1, 3))
  print('-'*100)
  print(f'{contador+1}° - {links[contador]}')

  var_titulo = drive_01.titulo_anuncio()
  df_dict['Titulo'].append(var_titulo)
  print('Titulo :                ', var_titulo)


  var_preco = drive_01.preco()
  df_dict['Preco'].append(var_preco)
  print('Preço R$:                ', var_preco)

  df_dict['Posicao'].append(contador + 1)

  var_vendas = drive_01.vendas()
  df_dict['Vendas_Anuncio'].append(var_vendas)
  print('Vendas deste Anuncio:  ', var_vendas)

  var_estoque = drive_01.estoque()
  df_dict['Estoque'].append(var_estoque)
  print('Estoque: ', var_estoque) 

  var_marca = drive_01.marca()
  df_dict['Marca'].append(var_marca)
  print('Marca:                   ', var_marca)

  var_desc = drive_01.desc()
  df_dict['Descricao'].append(var_desc)
  print('Descricao:                   ', var_desc)

  df_dict['Link'].append(links[contador])

df_01 = pd.DataFrame(df_dict)
df_01.to_excel(f'Tomada_Macho_{data}.xlsx')
df_01